<a href="https://colab.research.google.com/github/Ekliipce/2015-2017-Bitcoin-analysis/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator



import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [11]:
#!unzip '/content/drive/MyDrive/data/deepfashion.zip' -d '/content/drive/MyDrive/data/data_unzip/'

#!unzip '/content/drive/MyDrive/data/data_unzip/img.zip' -d '/content/drive/MyDrive/data/img/'


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000027.jpg  
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000028.jpg  
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000029.jpg  
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000030.jpg  
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000031.jpg  
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000032.jpg  
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000033.jpg  
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000034.jpg  
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000035.jpg  
  inflating: /content/drive/MyDrive/data/img/img/Woven_Dolphin_Shorts/img_00000036.jpg  
  inflating: /content/drive/MyDr

In [3]:
!find '/content/drive/MyDrive/data/img/img' -type d | wc -l

341


In [4]:
#path_base_data = "../data/"
path_base_data ='/content/drive/MyDrive/data'
path_list_category = "list_category_cloth.txt"
print(os.path.join(path_base_data, path_list_category))


with open(os.path.join(path_base_data, path_list_category)) as f:
    categories = f.readlines()
    categories = list(map(lambda x: x.split()[0], categories))
    categories= categories[2:]

for line in categories:
    print(line)


/content/drive/MyDrive/data/list_category_cloth.txt
Anorak
Blazer
Blouse
Bomber
Button-Down
Cardigan
Flannel
Halter
Henley
Hoodie
Jacket
Jersey
Parka
Peacoat
Poncho
Sweater
Tank
Tee
Top
Turtleneck
Capris
Chinos
Culottes
Cutoffs
Gauchos
Jeans
Jeggings
Jodhpurs
Joggers
Leggings
Sarong
Shorts
Skirt
Sweatpants
Sweatshorts
Trunks
Caftan
Cape
Coat
Coverup
Dress
Jumpsuit
Kaftan
Kimono
Nightdress
Onesie
Robe
Romper
Shirtdress
Sundress


In [5]:
print(f"number of categories: {len(categories)}")

number of categories: 50


In [6]:
path_category_img = "list_category_img.txt"

def handle_category_img(file):
    name = file.split()[0]
    name = name.split("/")
    if (len(name) >= 2):
        return name[1]

with open(os.path.join(path_base_data, path_category_img)) as f:
    category_img = f.readlines()
    category_img = list(map(handle_category_img, category_img))
    category_img = np.unique(category_img[2:])

for line in category_img:
    print(line)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Boxy_V-Neck_Top
Boxy_Vented_Tank
Boxy_Wide_Neck_Tee
Boxy_Winged_Tee
Boxy_Woven_Moto_Jacket
Boxy_Woven_Striped_Top
Boyfriend_Graphic_Tee
Boys_Aint_Loyal_Graphic_Tee
Boyz_II_Men_Muscle_Tee
Braided-Back_Abstract_Maxi_Dress
Braided-Strap_Tour_Graphic_Tank
Braided_Crew_Neck_Sweater
Braided_Halter_Top
Braided_Racerback_Athletic_Tank
Braided_Strap_Maxi_Dress
Braided_T-Back_Tank
Breton_Stripe_A-Line_Dress
Breton_Stripe_Henley
Broad_Stripe_Knit_Tee
Brocade_Floral_Peplum_Top
Brooklyn_Nets_Athletic_Shorts
Brooklyn_Nets_Baseball_Tee
Brooklyn_Nets_Graphic_Tank
Brooklyn_Nets_Jersey_Tank_Top
Brooklyn_Nets_Jersey_Top
Brooklyn_Nets_Logo_Hoodie
Brooklyn_Nets_Logo_Sweatpants
Brushed_Knit_Fringe_Top
Brushstroke_Print_Belted_Shift_Dress
Brushstroke_Print_Shirt_Dress
Brushstroke_Print_Sweatpants
Brushstroke_Stripe_Deep-V_Jumpsuit
Bull_Graphic_Muscle_Tee
Burnout_Baseball_Tee
Burnout_Crew_Neck_Tee
Burnout_Cutout-Back_Tank
Burnout_Ge

In [7]:

class_mapping = {}

for cat in categories:
    for name in category_img:
        if (cat in name):
            class_mapping[name] = cat

class_mapping

{'Hooded_Cotton_Canvas_Anorak': 'Anorak',
 'Abstract_Print_Draped_Blazer': 'Blazer',
 'Asymmetrical_Open-Front_Blazer': 'Blazer',
 'Boxy_Buttoned_Blazer': 'Blazer',
 'Button-Front_Blazer': 'Blazer',
 'Classic_Boyfriend_Blazer': 'Blazer',
 'Classic_Cuffed_Blazer': 'Blazer',
 'Classic_Denim_Blazer': 'Blazer',
 'Classic_One-Button_Blazer': 'Blazer',
 'Classic_Open-Front_Blazer': 'Blazer',
 'Classic_Single-Button_Blazer': 'Blazer',
 'Classic_Two-Button_Blazer': 'Blazer',
 'Classic_Woven_Blazer': 'Blazer',
 'Collarless_Draped-Front_Blazer': 'Blazer',
 'Collarless_Faux_Leather-Sleeve_Blazer': 'Blazer',
 'Collarless_One-Button_Blazer': 'Blazer',
 'Collarless_Open-Front_Blazer': 'Blazer',
 'Collarless_Woven_Blazer': 'Blazer',
 'Crepe_Shawl_Collar_Blazer': 'Blazer',
 'Cropped_Open-Front_Blazer': 'Blazer',
 'Cuffed-Sleeve_Boyfriend_Blazer': 'Blazer',
 'Cuffed-Sleeve_Knit_Blazer': 'Blazer',
 'Cuffed_Knit_Blazer': 'Blazer',
 'Double-Breasted_Blazer': 'Blazer',
 'Draped_Collar_Blazer': 'Blazer',
 '

In [8]:

v = class_mapping.values()
v = np.unique(list(v))

def remove_unpresent(v, categories):
  unpresent = []
  for cat in categories:
      if (cat not in v):
          unpresent.append(cat)
  for cat in unpresent:
    categories.remove(cat)
  return categories

remove_unpresent(v, categories)


['Anorak',
 'Blazer',
 'Blouse',
 'Bomber',
 'Button-Down',
 'Cardigan',
 'Flannel',
 'Halter',
 'Henley',
 'Hoodie',
 'Jacket',
 'Jersey',
 'Parka',
 'Peacoat',
 'Poncho',
 'Sweater',
 'Tank',
 'Tee',
 'Top',
 'Turtleneck',
 'Capris',
 'Chinos',
 'Culottes',
 'Cutoffs',
 'Gauchos',
 'Jeans',
 'Jeggings',
 'Jodhpurs',
 'Joggers',
 'Leggings',
 'Sarong',
 'Shorts',
 'Skirt',
 'Sweatpants',
 'Sweatshorts',
 'Trunks',
 'Caftan',
 'Coat',
 'Coverup',
 'Dress',
 'Jumpsuit',
 'Kaftan',
 'Kimono',
 'Onesie',
 'Robe',
 'Romper']

In [9]:
len(categories)

46

In [10]:
class CustomDirectoryIterator(DirectoryIterator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.label_names = np.array([class_mapping[self.filenames[i].split(os.path.sep)[-2]] for i in range(len(self.filenames))])
        self.class_indices = {v: k for k, v in enumerate(np.unique(self.label_names))}
        self.classes = np.array([self.class_indices[label] for label in self.label_names])
        self.num_classes = len(self.class_indices)
        print(self.classes)
        print(self.class_indices)

datagen = ImageDataGenerator(rescale=1./255)

generator = CustomDirectoryIterator(
    directory=os.path.join(path_base_data, "img/img"),
    image_data_generator=datagen,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=True
)

Found 15251 images belonging to 340 classes.
[23 23 23 ... 18 18 18]
{'Blazer': 0, 'Blouse': 1, 'Cardigan': 2, 'Culottes': 3, 'Cutoffs': 4, 'Dress': 5, 'Hoodie': 6, 'Jacket': 7, 'Jeans': 8, 'Jeggings': 9, 'Jersey': 10, 'Joggers': 11, 'Jumpsuit': 12, 'Kimono': 13, 'Leggings': 14, 'Poncho': 15, 'Romper': 16, 'Shorts': 17, 'Skirt': 18, 'Sweater': 19, 'Sweatpants': 20, 'Sweatshorts': 21, 'Tank': 22, 'Tee': 23, 'Top': 24}


In [11]:
generator.class_indices

{'Blazer': 0,
 'Blouse': 1,
 'Cardigan': 2,
 'Culottes': 3,
 'Cutoffs': 4,
 'Dress': 5,
 'Hoodie': 6,
 'Jacket': 7,
 'Jeans': 8,
 'Jeggings': 9,
 'Jersey': 10,
 'Joggers': 11,
 'Jumpsuit': 12,
 'Kimono': 13,
 'Leggings': 14,
 'Poncho': 15,
 'Romper': 16,
 'Shorts': 17,
 'Skirt': 18,
 'Sweater': 19,
 'Sweatpants': 20,
 'Sweatshorts': 21,
 'Tank': 22,
 'Tee': 23,
 'Top': 24}

In [12]:
def model(nbr_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    # Geler les couches du modèle de base pour l'entraînement
    for layer in base_model.layers:
        layer.trainable = False

    # Ajouter les couches personnalisées
    model = tf.keras.models.Sequential([
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(nbr_classes, activation='softmax')
    ])

    return model


In [13]:
model = model(generator.num_classes)
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 25)                6425      
                                                                 
Total params: 21,143,897
Trainable params: 6,429,209
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(generator, epochs=10)

Epoch 1/10
  5/477 [..............................] - ETA: 3:11:22 - loss: 9.7518 - accuracy: 0.0688

KeyboardInterrupt: ignored